In [1]:
import os
import asyncio
from dotenv import load_dotenv
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, From, To, Content, Email
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict

In [2]:
load_dotenv(override=True)

True

In [8]:
def send_test_email():
    sg = SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("soroushav85@gmail.com")
    to = To('soroush.alahvaisi@studenti.unime.it')
    content = Content('text/plain', content= "Hahahahahahaha new")
    mail = Mail(from_email=from_email, to_emails=to, subject="Test email", plain_text_content=content)
    response = sg.send(message=mail)
    print(response.status_code)

send_test_email()


202


In [9]:
### Agent Instructions (system promps)

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [10]:
names = ["Professional Sales Agent", "Engaging Sales Agent", "Busy Sales Agent"]
instructions = [instructions1, instructions2, instructions3]
sales_agents = []
for name, instruction in zip(names, instructions):
    sales_agents.append(Agent(name=name, instructions=instruction, model='gpt-4o-mini'))

sales_agent1, sales_agent2, sales_agent3 = sales_agents

In [11]:
result = Runner.run_streamed(sales_agent1, 'Write a cold sales email')

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in simplifying the SOC 2 compliance process through innovative AI-powered solutions.

Achieving and maintaining SOC 2 compliance can be a daunting task, often requiring extensive documentation and continuous monitoring. ComplAI is designed to streamline this process, transforming the way organizations prepare for audits by automating key tasks, tracking compliance metrics in real-time, and providing actionable insights.

**Key Benefits of ComplAI:**
- **Automated Documentation:** Reduce the manual effort involved in maintaining compliance documents.
- **Continuous Monitoring:** Stay proactive with real-time compliance tracking.
- **Audit Readiness:** Prepare seamlessly for audits and minimize last-minute stress.

I would love to schedule a brief call to discuss how ComplAI can specifically bene

In [18]:
message = "write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [ result.final_output for result in results]
all_mails = ""
for output in outputs:
    all_mails = all_mails + f"{output}\n\n"
    # print(f"{output}\n\n")
all_mails

"Subject: Simplify Your SOC 2 Compliance Process with ComplAI\n\nHi [Recipient's Name],\n\nI hope this message finds you well.\n\nI'm [Your Name], and I represent ComplAI, a leader in simplifying SOC 2 compliance for organizations like yours. Navigating the complexities of compliance can be a daunting task, but our AI-powered SaaS tool is designed to streamline the entire process, from preparation to audit.\n\nWith ComplAI, you can:\n\n- **Automate Documentation:** Reduce time spent on manual documentation by more than 50%.\n- **Stay Current:** Receive real-time updates on compliance requirements, ensuring you're always audit-ready.\n- **Enhance Security:** Identify and mitigate potential risks proactively with our advanced analytics.\n\nI would love the opportunity to discuss how our solution can help your team save time and mitigate compliance risks. Are you available for a quick call next week?\n\nThank you for considering ComplAI as a partner in your compliance journey.\n\nBest reg

In [15]:
sales_picker = Agent(
    name="sales_picker", 
    instructions="You pick the best cold sales email from the given options. \
    Imagine you are a customer and pick the one you are most likely to respond to. \
    Do not give an explanation; reply with the selected email only.",
    model='gpt-4o-mini'
)

In [19]:
with trace("Best Email"):
    result = await Runner.run(sales_picker, all_mails)
    print(result.final_output)

Subject: Is Your SOC2 Compliance Giving You Nightmares? 😱

Hey [First Name],

Ever feel like getting SOC2 compliant is like trying to fit a giraffe into a Smart Car? 🚗🦒

You’re not alone! At ComplAI, we know that audits can feel like an impending storm cloud—complete with thunder, lightning, and the occasional existential crisis. That’s why we’ve brewed up a special concoction of AI magic to sprinkle a little ease on your SOC2 compliance journey.

Our tool helps you:
- Navigate the labyrinth of compliance with the grace of a gazelle (or at least a well-coordinated squirrel).
- Prepare for audits without the caffeine-fueled panic attacks.
- Save time, money, and maybe a few hair follicles! 👨‍🦲

Curious to see how we can turn your compliance woes into ‘wow’ moments? Let’s schedule a quick chat. I promise I’ll even bring virtual donuts! 🍩

Looking forward to hearing from you!

Best,  
[Your Name]  
Your Compliance Cheerleader at ComplAI  
[Your Phone Number]  
[Your LinkedIn]  


In [20]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("soroushav85@gmail.com")
    to = To('soroush.alahvaisi@studenti.unime.it')
    content = Content('text/plain', content=body)
    mail = Mail(from_email=from_email, to_emails=to, subject="Sales email", plain_text_content=content)
    response = sg.send(message=mail)
    return {"status": "success"}

In [21]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10cf604a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [24]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name='sales_agent1', tool_description=description)
tool2 = sales_agent2.as_tool(tool_name='sales_agent2', tool_description=description)
tool3 = sales_agent3.as_tool(tool_name='sales_agent3', tool_description=description)

tools = [tool1, tool2, tool3, send_email]
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10e257f60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b26c040>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='Write 

In [25]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

sales_manager = Agent(name='sales_manager', instructions=instructions, tools=tools, model='gpt-4o-mini')

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)
    print(result)

RunResult:
- Last agent: Agent(name="sales_manager", ...)
- Final output (str):
    The email has been successfully sent to the CEO. If you need any further assistance or follow-up strategies, feel free to ask!
- 10 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [26]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email Subject Writer", instructions=subject_instructions, model="gpt-4o-mini")
html_converter = Agent(name="Html email body converter", instructions=html_instructions, model="gpt-4o-mini")

subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for an email")
html_converter_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to a html body")

In [27]:
@function_tool
def send_html_email(body: str, subject: str):
    """ Send out an email with the given body to all sales prospects """
    sg = SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("soroushav85@gmail.com")
    to = To('soroush.alahvaisi@studenti.unime.it')
    content = Content('text/html', content=body)
    mail = Mail(from_email=from_email, to_emails=to, subject=subject, plain_text_content=content)
    response = sg.send(message=mail)
    return {"status": "success"}

In [28]:
tools = [ subject_tool, html_converter_tool, send_html_email ]

In [29]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

email_agent = Agent(
    name='email_agent',
    instructions=instructions,
    tools=tools,
    model='gpt-4o-mini',
    handoff_description="Convert an email to html and send it"
)


In [30]:
tools = [ tool1, tool2, tool3 ]
handoffs = [email_agent]


In [ ]:

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'email_agent' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

sales_manager = Agent(
    name="sales_manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)